In [30]:
# Imports
import pandas as pd
import numpy as np
import pycountry

In [31]:
# ISO Countries names
iso2_to_name = {c.alpha_2: c.name for c in pycountry.countries}
iso3_to_name = {c.alpha_3: c.name for c in pycountry.countries}
name_to_iso2 = {c.name: c.alpha_2 for c in pycountry.countries}
iso2_to_iso3 = {c.alpha_2: c.alpha_3 for c in pycountry.countries}

In [32]:
# Data frame with the total number of patents granted per country/year
df_pat = pd.read_csv(r'total_patent_grants.csv', header=7).fillna(0)
df_pat = df_pat.rename(columns={'Origin (Code)': 'iso_2'})
df_pat['Name'] = df_pat['iso_2'].map(iso2_to_name)
df_pat

,Origin,iso_2,Office,Type,1980,1981,1982,1983,1984,1985,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Name
0,Albania,AL,Total,Total,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,3.0,12.0,6.0,0.0,12.0,4.0,0.0,7.0,Albania
1,Algeria,DZ,Total,Total,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,79.0,64.0,85.0,35.0,34.0,49.0,104.0,Algeria
2,Andorra,AD,Total,Total,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,10.0,11.0,12.0,14.0,18.0,Andorra
3,Angola,AO,Total,Total,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49.0,Angola
4,Argentina,AR,Total,Total,1591.0,820.0,740.0,538.0,513.0,0.0,...,410.0,407.0,375.0,377.0,355.0,290.0,368.0,439.0,501.0,Argentina
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,Yemen,YE,Total,Total,0.0,0.0,0.0,0.0,0.0,0.0,...,11.0,8.0,2.0,0.0,24.0,9.0,24.0,6.0,1.0,Yemen
170,Yugoslavia,YU,Total,Total,79.0,95.0,55.0,126.0,157.0,207.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
171,Zaire,ZR,Total,Total,3.0,8.0,6.0,6.0,9.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
172,Zambia,ZM,Total,Total,0.0,0.0,2.0,0.0,0.0,10.0,...,2.0,6.0,4.0,3.0,12.0,5.0,159.0,4.0,0.0,Zambia


In [33]:
# Missing names in Patents
miss_pat = df_pat.fillna('-')
miss_pat = miss_pat[miss_pat.Name == '-']
miss_pat

,Origin,iso_2,Office,Type,1980,1981,1982,1983,1984,1985,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Name
43,Czechoslovakia,CS,Total,Total,6768.0,5447.0,6116.0,6200.0,6267.0,7.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-
62,German Democratic Republic,DD,Total,Total,4455.0,5713.0,4125.0,10709.0,11402.0,11487.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-
108,Namibia,0,Total,Total,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,13.0,16.0,14.0,4.0,8.0,9.0,0.0,-
145,Soviet Union,SU,Total,Total,92909.0,96544.0,89305.0,72635.0,62755.0,73282.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-
170,Yugoslavia,YU,Total,Total,79.0,95.0,55.0,126.0,157.0,207.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-
171,Zaire,ZR,Total,Total,3.0,8.0,6.0,6.0,9.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-


In [34]:
# Replace missing country name in Patents
missing = {
    'German Democratic Republic': 'Germany',
    #'Soviet Union': 'Russian Federation'
}
for k in missing.keys():
    i = df_pat[df_pat.Origin == k].index[0]
    df_pat.iloc[i, 46] = missing[k]

In [35]:
# Group values by names
df_pat = df_pat.drop(columns=['Origin', 'iso_2', 'Office', 'Type']).dropna()
df_pat = df_pat.groupby(['Name'], as_index=False).sum()

In [36]:
# Convert columns into rows
df = df_pat.melt(id_vars=['Name'], var_name='Year', value_name='Patents')
df

,Name,Year,Patents
0,Albania,1980,0.0
1,Algeria,1980,0.0
2,Andorra,1980,0.0
3,Angola,1980,0.0
4,Argentina,1980,1591.0
...,...,...,...
7051,"Venezuela, Bolivarian Republic of",2021,0.0
7052,Viet Nam,2021,227.0
7053,Yemen,2021,1.0
7054,Zambia,2021,0.0


In [37]:
# Data frame with the population per country/year
df_pop = pd.read_csv(r'population.csv', header=2).drop(
    columns=['Indicator Name', 'Indicator Code']).rename(
        columns={'Country Code': 'iso_3'})
df_pop['Name'] = df_pop['iso_3'].map(iso3_to_name)
df_pop = df_pop.drop(
    columns=[
        'Unnamed: 66',
        'Country Name',
        'iso_3'
        ]
).dropna()
df_pop

,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Name
0,54608.0,55811.0,56682.0,57475.0,58178.0,58782.0,59291.0,59522.0,59471.0,59330.0,...,102880.0,103594.0,104257.0,104874.0,105439.0,105962.0,106442.0,106585.0,106537.0,Aruba
2,8622466.0,8790140.0,8969047.0,9157465.0,9355514.0,9565147.0,9783147.0,10010030.0,10247780.0,10494489.0,...,31541209.0,32716210.0,33753499.0,34636207.0,35643418.0,36686784.0,37769499.0,38972230.0,40099462.0,Afghanistan
4,5357195.0,5441333.0,5521400.0,5599827.0,5673199.0,5736582.0,5787044.0,5827503.0,5868203.0,5928386.0,...,26147002.0,27128337.0,28127721.0,29154746.0,30208628.0,31273533.0,32353588.0,33428486.0,34503774.0,Angola
5,1608800.0,1659800.0,1711319.0,1762621.0,1814135.0,1864791.0,1914573.0,1965598.0,2022272.0,2081695.0,...,2895092.0,2889104.0,2880703.0,2876101.0,2873457.0,2866376.0,2854191.0,2837849.0,2811666.0,Albania
6,9443.0,10216.0,11014.0,11839.0,12690.0,13563.0,14546.0,15745.0,17079.0,18449.0,...,71367.0,71621.0,71746.0,72540.0,73837.0,75013.0,76343.0,77700.0,79034.0,Andorra
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260,113335.0,116820.0,120163.0,123416.0,126582.0,129789.0,132976.0,135752.0,138154.0,140457.0,...,199939.0,201757.0,203571.0,205544.0,207630.0,209701.0,211905.0,214929.0,218764.0,Samoa
262,5542459.0,5646668.0,5753386.0,5860197.0,5973803.0,6097298.0,6228430.0,6368014.0,6515904.0,6673981.0,...,26984002.0,27753304.0,28516545.0,29274002.0,30034389.0,30790513.0,31546691.0,32284046.0,32981641.0,Yemen
263,16520441.0,16989464.0,17503133.0,18042215.0,18603097.0,19187194.0,19789771.0,20410677.0,21050540.0,21704214.0,...,53873616.0,54729551.0,55876504.0,56422274.0,56641209.0,57339635.0,58087055.0,58801927.0,59392255.0,South Africa
264,3119430.0,3219451.0,3323427.0,3431381.0,3542764.0,3658024.0,3777680.0,3901288.0,4029173.0,4159007.0,...,15234976.0,15737793.0,16248230.0,16767761.0,17298054.0,17835893.0,18380477.0,18927715.0,19473125.0,Zambia


In [38]:
# Convert columns into rows
df_temp = df_pop.melt(
    id_vars=['Name'],
    var_name='Year',
    value_name='Population'
)
df_temp

,Name,Year,Population
0,Aruba,1960,54608.0
1,Afghanistan,1960,8622466.0
2,Angola,1960,5357195.0
3,Albania,1960,1608800.0
4,Andorra,1960,9443.0
...,...,...,...
13263,Samoa,2021,218764.0
13264,Yemen,2021,32981641.0
13265,South Africa,2021,59392255.0
13266,Zambia,2021,19473125.0


In [39]:
# Merge the data frames
df = pd.merge(df, df_temp, on=['Name', 'Year'], how='left')
df

,Name,Year,Patents,Population
0,Albania,1980,0.0,2671997.0
1,Algeria,1980,0.0,18739378.0
2,Andorra,1980,0.0,35611.0
3,Angola,1980,0.0,8330047.0
4,Argentina,1980,1591.0,28024803.0
...,...,...,...,...
7051,"Venezuela, Bolivarian Republic of",2021,0.0,28199867.0
7052,Viet Nam,2021,227.0,97468029.0
7053,Yemen,2021,1.0,32981641.0
7054,Zambia,2021,0.0,19473125.0


In [40]:
# Data frame with the GDP per country/year
df_gdp = pd.read_csv(r'gdp.csv', header=4).drop(
    columns=['Indicator Name', 'Indicator Code']).rename(
        columns={'Country Code': 'iso_3'}).fillna(0)
df_gdp['Name'] = df_gdp['iso_3'].map(iso3_to_name)
df_gdp = df_gdp.drop(columns=['Country Name', 'iso_3']).dropna()
df_gdp

,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Name
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,26515.678080,26942.307976,28421.386493,28451.273745,29326.708058,30220.594523,31650.760537,24487.863560,29342.100858,Aruba
2,62.369375,62.443703,60.950364,82.021738,85.511073,105.243196,143.103233,167.165675,134.012768,134.250360,...,651.987862,628.146804,592.476537,520.252064,530.149831,502.056771,500.522664,516.866552,368.754614,Afghanistan
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,5101.983876,5059.080441,3100.830685,1709.515534,2283.214233,2487.500996,2142.238757,1603.993477,1953.533757,Angola
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,4413.062005,4578.633208,3952.802538,4124.055390,4531.019374,5287.663694,5396.215864,5332.160475,6492.872012,Albania
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,44750.435680,45682.246231,38885.376014,39932.164487,40632.484393,42903.443579,41327.502031,37207.493861,42137.327271,Andorra
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,3989.860174,3948.801128,4048.513876,4105.834468,4261.653789,4188.988532,4308.272753,4042.751224,3857.318464,Samoa
262,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1497.747941,1557.601406,1488.416482,1069.817122,893.716573,701.714878,0.000000,0.000000,0.000000,Yemen
263,529.561923,543.042224,560.699395,601.599951,642.688431,681.131112,718.118179,775.152812,813.508497,887.246703,...,7441.230854,6965.137897,6204.929901,5735.066787,6734.475153,7048.522211,6688.787271,5741.643129,7055.044776,South Africa
264,228.567399,216.274674,208.562685,209.453362,236.941713,296.022427,334.672528,350.653425,398.557506,472.640293,...,1840.320553,1724.576220,1307.909649,1249.923143,1495.752138,1475.204538,1268.120941,956.831364,1137.343633,Zambia


In [41]:
# Convert columns into rows
df_temp = df_gdp.melt(
    id_vars=['Name'],
    var_name='Year',
    value_name='GDP'
)
df_temp

,Name,Year,GDP
0,Aruba,1960,0.000000
1,Afghanistan,1960,62.369375
2,Angola,1960,0.000000
3,Albania,1960,0.000000
4,Andorra,1960,0.000000
...,...,...,...
13325,Samoa,2021,3857.318464
13326,Yemen,2021,0.000000
13327,South Africa,2021,7055.044776
13328,Zambia,2021,1137.343633


In [42]:
# Merge the data frames
df = pd.merge(df, df_temp, on=['Name', 'Year'], how='left')
df

,Name,Year,Patents,Population,GDP
0,Albania,1980,0.0,2671997.0,0.000000
1,Algeria,1980,0.0,18739378.0,2259.753810
2,Andorra,1980,0.0,35611.0,12535.904800
3,Angola,1980,0.0,8330047.0,711.941169
4,Argentina,1980,1591.0,28024803.0,2746.207484
...,...,...,...,...,...
7051,"Venezuela, Bolivarian Republic of",2021,0.0,28199867.0,0.000000
7052,Viet Nam,2021,227.0,97468029.0,3756.489121
7053,Yemen,2021,1.0,32981641.0,0.000000
7054,Zambia,2021,0.0,19473125.0,1137.343633


In [43]:
# Add Patents per 100 thousand inhabitants
df['Pat_100k'] = df.Patents.div(100000)

# Add Iso_2
df['iso_2'] = df['Name'].map(name_to_iso2)

# Add Iso_3
df['iso_3'] = df['iso_2'].map(iso2_to_iso3)

In [44]:
# Save the data frame to CSV
df.to_csv('dataset.csv')